# M-Shots Learning

In this notebook, we'll explore small prompt engineering techniques and recommendations that will help us elicit responses from the models that are better suited to our needs.

In [8]:
!pip uninstall dataclasses



In [9]:
!pip install --upgrade openai python-dotenv


  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)


In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install dotenv

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × pip subprocess to install backend dependencies did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
        Using cached distribute-0.7.3.zip (145 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'error'
        error: subprocess-exited-with-error
      
        Ã— Preparing metadata (pyproject.toml) did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [6 lines of output]
            usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
               or: setup.py --help [cmd1 cmd2 ...]
               or: setup.py --help-commands
               or: setup.py cmd --help
      
            

In [1]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

# Formatting the answer with Few Shot Samples.

To obtain the model's response in a specific format, we have various options, but one of the most convenient is to use Few-Shot Samples. This involves presenting the model with pairs of user queries and example responses.

Large models like GPT-3.5 respond well to the examples provided, adapting their response to the specified format.

Depending on the number of examples given, this technique can be referred to as:
* Zero-Shot.
* One-Shot.
* Few-Shots.

With One Shot should be enough, and it is recommended to use a maximum of six shots. It's important to remember that this information is passed in each query and occupies space in the input prompt.



In [2]:
# Function to call the model.
def return_OAIResponse(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=1,
        )

    return (response.choices[0].message.content)

In this zero-shots prompt we obtain a correct response, but without formatting, as the model incorporates the information he wants.

In [3]:
#zero-shot
context_user = [
    {'role':'system', 'content':'You are an expert in F1.'}
]
print(return_OAIResponse("Who won the F1 2010?", context_user))

Sebastian Vettel won the F1 2010 championship driving for Red Bull Racing.


For a model as large and good as GPT 3.5, a single shot is enough to learn the output format we expect.


In [4]:
#one-shot
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2000 f1 championship?
     Driver: Michael Schumacher.
     Team: Ferrari."""}
]
print(return_OAIResponse("Who won the F1 2011?", context_user))

The 2011 F1 championship was won by Sebastian Vettel, driving for Red Bull Racing.


Smaller models, or more complicated formats, may require more than one shot. Here a sample with two shots.

In [5]:
#Few shots
context_user = [
    {'role':'system', 'content':
     """You are an expert in F1.

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

The 2006 F1 Championship was won by Fernando Alonso, driving for the Renault F1 Team.


In [6]:
print(return_OAIResponse("Who won the F1 2019?", context_user))

The 2019 F1 World Championship was won by Lewis Hamilton driving for Mercedes.


We've been creating the prompt without using OpenAI's roles, and as we've seen, it worked correctly.

However, the proper way to do this is by using these roles to construct the prompt, making the model's learning process even more effective.

By not feeding it the entire prompt as if they were system commands, we enable the model to learn from a conversation, which is more realistic for it.

In [7]:
#Recomended solution
context_user = [
    {'role':'system', 'content':'You are and expert in f1.\n\n'},
    {'role':'user', 'content':'Who won the 2010 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Sebastian Vettel. \nTeam: Red Bull. \nPoints: 256. """},
    {'role':'user', 'content':'Who won the 2009 f1 championship?'},
    {'role':'assistant', 'content':"""Driver: Jenson Button. \nTeam: BrawnGP. \nPoints: 95. """},
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton. 
Team: Mercedes. 
Points: 413.


We could also address it by using a more conventional prompt, describing what we want and how we want the format.

However, it's essential to understand that in this case, the model is following instructions, whereas in the case of use shots, it is learning in real-time during inference.

In [8]:
context_user = [
    {'role':'system', 'content':"""You are and expert in f1.
    You are going to answer the question of the user giving the name of the rider,
    the name of the team and the points of the champion, following the format:
    Drive:
    Team:
    Points: """
    }
]

print(return_OAIResponse("Who won the F1 2019?", context_user))

Driver: Lewis Hamilton
Team: Mercedes
Points: 413


In [9]:
context_user = [
    {'role':'system', 'content':
     """You are classifying .

     Who won the 2010 f1 championship?
     Driver: Sebastian Vettel.
     Team: Red Bull Renault.

     Who won the 2009 f1 championship?
     Driver: Jenson Button.
     Team: BrawnGP."""}
]
print(return_OAIResponse("Who won the F1 2006?", context_user))

Driver: Fernando Alonso.
Team: Renault.


Few Shots for classification.


In [10]:
context_user = [
    {'role':'system', 'content':
     """You are an expert in reviewing product opinions and classifying them as positive or negative.

     It fulfilled its function perfectly, I think the price is fair, I would buy it again.
     Sentiment: Positive

     It didn't work bad, but I wouldn't buy it again, maybe it's a bit expensive for what it does.
     Sentiment: Negative.

     I wouldn't know what to say, my son uses it, but he doesn't love it.
     Sentiment: Neutral
     """}
]
print(return_OAIResponse("I'm not going to return it, but I don't plan to buy it again.", context_user))

Sentiment: Neutral


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=200
    )
    return response.choices[0].message.content

context = [
    {'role': 'system', 'content': 'You are an expert in luxury cars, helping users decide on the perfect car for their needs.'},
    {'role': 'user', 'content': "I want to get my mum a luxury car for her birthday. Can you help me choose?"}
]

response = get_completion_from_messages(context)
print(response)

# I asked the AI to aid me in picking a luxury car for my mums birthday and although it gave me a fair idea of what i needed it wasnt as detailed as i would have liked


Of course! I'd be happy to help you choose a luxury car for your mum's birthday. To provide you with the best recommendation, could you please provide me with some more information? 

1. What is your budget for the luxury car?
2. What are your mum's preferences in terms of car size (compact, midsize, SUV, etc.)?
3. Does she have a preference for a specific luxury car brand?
4. Are there any specific features or amenities that are important to her, such as advanced safety features, technology, comfort, or performance?
5. Will she be using the car for daily commuting, long road trips, or other specific purposes?

With this information, I can suggest some luxury cars that would be a great fit for your mum's birthday gift.


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=200
    )
    return response.choices[0].message.content

context = [
    {'role': 'system', 'content': 'You are a football analyst with expertise in African football, using data from the 2024 AFCON qualifiers to predict the potential winner of the 2025 AFCON.'},
    {'role': 'user', 'content': 'Based on the 2024 AFCON qualifiers, which team is most likely to win the 2025 AFCON?'},
    {'role': 'assistant', 'content': 'Based on the qualifiers, Morocco is a strong contender with a solid team performance and consistent wins.'},
    {'role': 'user', 'content': 'What about Senegal and Nigeria?'},
    {'role': 'assistant', 'content': 'Senegal and Nigeria are both strong teams. Senegal has a highly cohesive squad with experienced players, while Nigeria boasts younger talent and attacking versatility.'},
    {'role': 'user', 'content': 'Considering all teams, who is the favorite to win?'}
]

response = get_completion_from_messages(context)
print(response)

#showcases the AI´s ability to use examples to make a prediction, however it was limited to the quality of examples given


Considering all teams, including Morocco, Senegal, and Nigeria, Senegal might be considered the favorite to win the 2025 AFCON due to their strong squad and recent performances. However, Morocco and Nigeria are also strong contenders and could potentially upset the odds.


In [ ]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY,
)

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=200
    )
    return response.choices[0].message.content

context = [
    {'role': 'system', 'content': 'You are a highly knowledgeable football analyst, specializing in African football, who provides detailed predictions based on data and performance from the 2024 AFCON qualifiers. Your responses are data-driven and include justifications for your predictions.'},
    {'role': 'user', 'content': 'Which team has the highest chances of winning the 2025 AFCON, and why?'},
    {'role': 'assistant', 'content': 'Morocco emerges as the top contender based on their performance in the 2024 qualifiers. They exhibited the highest goal difference, a stable defense, and star players like Hakim Ziyech and Achraf Hakimi playing at their peak. Their tactical adaptability is unmatched in recent tournaments.'},
    {'role': 'user', 'content': 'Can you include statistics for their performance?'},
    {'role': 'assistant', 'content': 'Morocco scored an average of 2.5 goals per game and conceded only 0.5 goals per match during the qualifiers. Their win rate stood at 80%, and they demonstrated strong team cohesion in critical matches.'},
    {'role': 'user', 'content': 'How do Senegal and Algeria compare?'},
    {'role': 'assistant', 'content': 'Senegal showed consistent performance with a balanced squad and a win rate of 70%. They boast players like Sadio Mane and Kalidou Koulibaly, bringing experience and reliability. Algeria, on the other hand, demonstrated a strong attack, scoring an average of 2 goals per game but struggled defensively, conceding 1.2 goals per game on average.'},
    {'role': 'user', 'content': 'Considering all factors, who is your prediction for the 2025 AFCON winner?'}
]

response = get_completion_from_messages(context)
print(response)

#Gave me an informed analysis on the next afcon winner by using data driven predictions with good judgement


Taking into account all factors, including performance in the qualifiers, squad strength, tactical prowess, and consistency, Morocco stands out as the favorite to win the 2025 AFCON. Their solid defense, potent attack, and star players give them a well-rounded edge over other contenders.
